In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from keras.datasets import mnist
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 문장 split.
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#### 토큰화 (Tokenization)
- 구두점이나 특수문자를 전부 제거하면 토큰이 의미를 잃어버리는 경우가 발생하기도 함.
    - 1.구두점이나 특수문자를 단순 제외하면 안되는 이유.
        - (1). 단어 자체에 구두점을 갖고 있는 경우 : ph.D, KT&G
        - (2). 특수문자가 의미를 가지고 있는 경우 : 17/05/31, $756
    - 2. 줄임말과 단어 내 띄어쓰기가 있는 경우.
        - 1) 줄임말 : I'm = I am
        - 2) 하나의 단어이지만 띄어쓰기가 있는 경우 : New York

In [2]:
# 예시 텍스트 설정.
text1 = '해보지 않으면 해낼 수 없다'
text2 = 'I can do it'
# 해당 텍스트를 토큰화.
# text_to_word_sequenc() : 모든 알파벳을 소문자로 변경. .,?! 등은 제거하나 '는 보존.
result1 = text_to_word_sequence(text1)
result2 = text_to_word_sequence(text2)

print(f'원본1 : {text1}')
print(f'토큰화1 : {result1}')
print('-'*52)
print(f'원본2 : {text2}')
# 대문자 I가 소문자로 변경.
print(f'토큰화2 : {result2}') 

원본1 : 해보지 않으면 해낼 수 없다
토큰화1 : ['해보지', '않으면', '해낼', '수', '없다']
----------------------------------------------------
원본2 : I can do it
토큰화2 : ['i', 'can', 'do', 'it']


In [3]:
# 문장 텍스트 설정.
docs = [
    '먼저 텍스트의 각 단어를 나누어 토큰화 합니다',
    '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다',
    '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다'
]

In [4]:
# 토큰화 함수를 통해 전처리를 한다.
# Tokenizer() : 가장 빈도가 높은 num_words=None 수 만큼의 단어만 선택하도록하는 Tokenizer 객체.
token = Tokenizer()

# 이에 맞게 단어 인덱스를 구축함.
token.fit_on_texts(docs)

In [5]:
# 전체 문장의 개수.
token.document_count

3

In [6]:
# 단어의 빈도수.
token.word_counts

OrderedDict([('먼저', 1),
             ('텍스트의', 2),
             ('각', 1),
             ('단어를', 1),
             ('나누어', 1),
             ('토큰화', 3),
             ('합니다', 1),
             ('단어로', 1),
             ('해야', 1),
             ('딥러닝에서', 2),
             ('인식됩니다', 1),
             ('한', 1),
             ('결과는', 1),
             ('사용할', 1),
             ('수', 1),
             ('있습니다', 1)])

In [7]:
# 각 단어가 몇개의 문장에 포함되어 있는지를 보여줌. 출력되는 순서는 랜덤.
token.word_docs

defaultdict(int,
            {'각': 1,
             '먼저': 1,
             '텍스트의': 2,
             '단어를': 1,
             '토큰화': 3,
             '나누어': 1,
             '합니다': 1,
             '단어로': 1,
             '인식됩니다': 1,
             '딥러닝에서': 2,
             '해야': 1,
             '한': 1,
             '결과는': 1,
             '사용할': 1,
             '있습니다': 1,
             '수': 1})

In [8]:
# 각 단어에 부여된 인덱스.
token.word_index

{'토큰화': 1,
 '텍스트의': 2,
 '딥러닝에서': 3,
 '먼저': 4,
 '각': 5,
 '단어를': 6,
 '나누어': 7,
 '합니다': 8,
 '단어로': 9,
 '해야': 10,
 '인식됩니다': 11,
 '한': 12,
 '결과는': 13,
 '사용할': 14,
 '수': 15,
 '있습니다': 16}

In [9]:
# 평가글.
docs = [
    '너무 재밌네요',
    '최고예요',
    '참 잘 만든 영화에요',
    '추천하고 싶은 영화입니다',
    '한번 더 보고 싶네요',
    '글쎄요',
    '별로에요',
    '생각보다 지루하네요',
    '연기가 어색해요',
    '재미없어요'
]

# 결과 데이터.(분류)
# 1 : 긍정. , 0 : 부정.
classes = np.array([1,1,1,1,1,0,0,0,0,0])

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)
print('-'*135)

# 각 문장을 토큰화 시킨 데이터를 단어 인덱스로 변환.
x = token.texts_to_sequences(docs)
print(x)
print('-'*135)

# 각 리스트의 데이터의 개수를 최대 개수로 통일.
# pad_sequences() : 문장의 길이를 maxlen 인자로 맞춰줌.
# 예를 들어 4로 지정했다면 4보다 짧은 문장은 0으로 채워서 4단어로 맞춰주고, 4보다 긴 문장은 4단어까지만 잘라냄.
padded_x = pad_sequences(x,  maxlen=4)
padded_x

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고': 13, '싶네요': 14, '글쎄요': 15, '별로에요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21}
---------------------------------------------------------------------------------------------------------------------------------------
[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13, 14], [15], [16], [17, 18], [19, 20], [21]]
---------------------------------------------------------------------------------------------------------------------------------------


array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [11, 12, 13, 14],
       [ 0,  0,  0, 15],
       [ 0,  0,  0, 16],
       [ 0,  0, 17, 18],
       [ 0,  0, 19, 20],
       [ 0,  0,  0, 21]])

In [10]:
# 단어의 수를 파악.
# 통상 to_categorical( ) 함수를 사용해 원-핫 인코딩 과정을 진행.
# 배열 맨 앞에 0으로 이루어진 인덱스가 추가되므로 단어 수보다 1이 더 많게 인덱스 숫자를 잡아주는 것에 유의.
word_size = len(token.word_index) + 1

In [11]:
# 학습 모델을 구성.
model = Sequential()

# Embedding : 많은 단어의 수를 줄임. : 공간 낭비를 줄이기 위해 단어 임베딩은 주어진 배열을 정해진 길이로 압축.
# word_size : 전체 단어의 수, 8 : 줄일 사이즈, input_length=4 : 입력층의 노드의 개수.
model.add(Embedding(word_size, 8, input_length=4))

# 1차원으로 변환.
# Flatten() : 다차원 배열을 1차원으로 바꿈.
model.add(Flatten())

# 출력층 생성.
# 이진 분류이므로 활성화 함수로 sigmoid 설정.
model.add(Dense(1, activation='sigmoid'))

# 컴파일.
# model.compile() : 모델 구성 후 compile() 메소드를 호출하여 모델 학습 과정을 설정.
# loss : 최적화 과정에서 최소화될 손실 함수를 설정하는 것으로, MSE(평균 제곱 오차)와 binary_crossentropy가 자주 사용됨.
# optimizer : 훈련 과정을 설정하는 것으로, Adam, SGD 등이 존재.
# metrics : 훈련을 모니터링을 위해 사용.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습.
model.fit(padded_x, classes, epochs=20, verbose=1 )

# 정확도.
print(f'Accuracy : model.evaluate(padded_x, classes)[1]')

Epoch 1/20
1/1 [==============================] - 1s 752ms/step - loss: 0.6985 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6962 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6940 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6918 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6895 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6873 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6851 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6829 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6806 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6784 - accuracy: 0.8000
Epoch 11/20
1/1 [====================